In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor, Compose
from torchvision.datasets import OxfordIIITPet
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, to_pil_image

import os
import tarfile
from torchvision.datasets import OxfordIIITPet
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

import os
import torch
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset

class OxfordPetDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images_dir = os.path.join(root_dir, "images")
        self.masks_dir = os.path.join(root_dir, "annotations", "trimaps")
        self.image_filenames = os.listdir(self.images_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_filename = self.image_filenames[idx]
        image_path = os.path.join(self.images_dir, image_filename)

        # 마스크 파일 이름은 이미지 파일 이름에서 확장자만 바꾸면 됩니다
        mask_filename = os.path.splitext(image_filename)[0] + ".png"
        mask_path = os.path.join(self.masks_dir, mask_filename)

        image = Image.open(image_path)
        mask = Image.open(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

# 데이터셋과 변환 정의
dataset = OxfordPetDataset(root_dir="/content/oxford-iiit-pet", transform=ToTensor())

# 첫 번째 이미지와 마스크 로드
image, mask = dataset[0]




# 이미지와 마스크를 시각화합니다
plt.subplot(1, 2, 1)
plt.imshow(image.permute(1, 2, 0))  # 이미지의 채널 순서를 변경합니다
plt.title('Image')
plt.subplot(1, 2, 2)
plt.imshow(mask.squeeze(0), cmap='gray')  # 첫 번째 차원을 제거합니다
plt.title('Mask')
plt.show()



class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )

        def pool_block():
            return nn.MaxPool2d(kernel_size=2, stride=2)

        def upsample_block(in_channels, out_channels):
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
                nn.ReLU(inplace=True)
            )

        self.encoder = nn.Sequential(
            conv_block(in_channels, 64),
            pool_block(),
            conv_block(64, 128),
            pool_block(),
            conv_block(128, 256),
            pool_block(),
            conv_block(256, 512),
            pool_block()
        )

        self.middle = conv_block(512, 1024)

        self.decoder = nn.Sequential(
            upsample_block(1024, 512),
            conv_block(512 + 512, 512),
            upsample_block(512, 256),
            conv_block(256 + 256, 256),
            upsample_block(256, 128),
            conv_block(128 + 128, 128),
            upsample_block(128, 64),
            conv_block(64 + 64, 64)
        )

        self.output_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.encoder[:2](x)
        enc2 = self.encoder[2:5](enc1)
        enc3 = self.encoder[5:8](enc2)
        enc4 = self.encoder[8:](enc3)

        middle = self.middle(enc4)

        dec1 = self.decoder[:2](middle)
        dec1 = torch.cat([dec1, enc4], dim=1)
        dec2 = self.decoder[2:4](dec1)
        dec2 = torch.cat([dec2, enc3], dim=1)
        dec3 = self.decoder[4:6](dec2)
        dec3 = torch.cat([dec3, enc2], dim=1)
        dec4 = self.decoder[6:8](dec3)
        dec4 = torch.cat([dec4, enc1], dim=1)

        out = self.output_conv(dec4)
        return out


c:\Users\L3352\.conda\envs\new2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\L3352\.conda\envs\new2\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\L3352\.conda\envs\new2\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '/content/oxford-iiit-pet\\images'

: 

In [ ]:
# 데이터셋의 샘플 수 확인
print("Dataset length:", len(dataset))

# 이미지와 마스크의 크기 확인
print("Image size:", image.size())

from PIL import Image
import numpy as np

# 정수형 리스트를 NumPy 배열로 변환
mask_array = np.array(mask)

# NumPy 배열을 PIL 이미지로 변환
mask_image = Image.fromarray(mask_array)

# PIL 이미지를 텐서로 변환
mask_tensor = to_tensor(mask_image)

print("Mask size:", mask_tensor.size())



# UNet 모델 구조 확인
model = UNet(in_channels=3, out_channels=1)  # 입력 채널 수와 출력 채널 수에 맞게 수정
print(model)

# 입력 이미지를 모델에 전달하여 출력 확인
output = model(image.unsqueeze(0))
print("Output size:", output.size())
